<a href="https://colab.research.google.com/github/aknip/Langchain-etc./blob/main/Langchain-Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain Agent

1. Run cells "Setup for all agents and apps"
2. Run one of the Agent sections / cells
3. Run Gradio app or CLI apps


# Setup for all agents and apps

In [116]:
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])

In [117]:
!pip install langchain langchainhub langchain_experimental google-search-results wikipedia openai gradio -q
# !pip install google-search-results wikipedia -q
%load_ext gradio

The gradio extension is already loaded. To reload it, use:
  %reload_ext gradio


In [ ]:
import json
import os
from getpass import getpass
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

In [118]:
from langchain.agents import Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper, SQLDatabase, WikipediaAPIWrapper
from langchain_experimental.sql import SQLDatabaseChain
from langchain.tools import WikipediaQueryRun
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool, StructuredTool
from langchain.schema.agent import AgentFinish
import os
import langchain
import openai
import textwrap
import gradio as gr
import random
import time

# Agent 1: Wikpedia, Search, Music DB

In [ ]:
# Download "Chinook" Music Sales DB
import os
import requests
import zipfile
fname = 'chinook.zip'
url = 'https://www.sqlitetutorial.net/wp-content/uploads/2018/03/' + fname
r = requests.get(url)
open(fname, 'wb').write(r.content)
zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

In [ ]:
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential']
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')
search = SerpAPIWrapper(serpapi_api_key=CREDS['SERP-API']['key']['credential'])
db = SQLDatabase.from_uri("sqlite:///chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about real-time events. You should ask targeted questions"
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to answer questions about a big picture or background of something."
    ),
    Tool(
        name="MusicSales",
        func=db_chain.run,
        description="useful for when you need to answer questions about mucis sales in a store. Should be strickly follow the tables info."
    )
]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant."),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()




# Agent 2: Calculate insurance premium

In [ ]:
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential']
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')

def calculate_insurance_premium(insured_sum: int, industry: str) -> float:
    """Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer."""
    premium = insured_sum*0.25
    return premium

calculation_tool = StructuredTool.from_function(calculate_insurance_premium, return_direct=False) # True return result without additional text

tools = [
    calculation_tool
]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant."),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()


# Gradio Chat App for Agent

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

#
# Gradio app
#

# Theming
theme = gr.themes.Default(
    primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
)
# Styling: Change max width
css = """
  .gradio-container {max-width: 800px!important}
  .vspacer1 {margin-top: 20px}
"""

with gr.Blocks(theme=theme, css=css) as demo:

    gr.Markdown("# Agent Chat", elem_classes="vspacer1")

    with gr.Tab("Chat"):
      # https://www.gradio.app/docs/chatbot

      chatbot = gr.Chatbot(bubble_full_width=False)
      msg = gr.Textbox()
      clear = gr.ClearButton([msg, chatbot])

      # Init agent and memory
      memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
      agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
      messages = [{"role": "assistant", "content": "How can I help you?"}]
      #chatbot.append(None, "How can I help you?")

      def ask(message, chat_history):
          chat_history.append((message, None))
          messages.append({"role": "user", "content": message})
          return "", chat_history

      def respond(chat_history):
          prompt = chat_history[-1][0] # get prompt from history (last entry)
          response = agent_executor.invoke({"input": prompt})
          msg = {"role": "assistant", "content": response["output"]}
          messages.append(msg)
          chat_history.append((None, response["output"]))
          print("\n\nMemory from response object:")
          print(textwrap.fill(str(response["chat_history"]), 80))
          return chat_history

      msg.submit(ask, [msg, chatbot], [msg, chatbot], queue=False).then(
        respond, chatbot, chatbot
      )

demo.launch(quiet=True, share=False, debug=True)

# CLI für agent_executor (Single Prompt)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False,  memory=memory)

langchain.debug = False

prompt = "Give me the premium for an insured sum of 10000000 for metal construcion industry"
# prompt = "Wie viele Menschen leben in den USA?"
# prompt = "How many people live in the US?"
# prompt = "What is the best selling song in the music store?"
# prompt = "What is the cheapest album in the music store? Tell me the price."
# prompt = "What is the best selling song starting with the letter 'F' in the music store? Tell me title and artist."
# prompt = "How is the weather now in Cologne? Please use Celsius as unit for temperature."
response = agent_executor.invoke({"input": prompt})

# Response-Objekte:
print()
print(textwrap.fill(str(response["input"]), 80))
print()
print(textwrap.fill(str(response["output"]), 80))
print()
chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
chat_history = chat_history_string.split("§§§ ")
msg_history = []
for chat_string in chat_history:
  tmp = chat_string.split("(content='")
  msg_history.append({"role": tmp[0], "content": tmp[1][:-2]})
for msg in msg_history:
  print(msg)

In [ ]:

print(textwrap.fill(str(response["input"]), 80))
print()
print(textwrap.fill(str(response["output"]), 80))
print()
chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
chat_history = chat_history_string.split("§§§ ")
msg_history = []
for chat_string in chat_history:
  tmp = chat_string.split("(content='")
  msg_history.append({"role": tmp[0], "content": tmp[1][:-2]})
for msg in msg_history:
  print(msg)

# CLI for agent_executor (Chatbot with memory)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
messages = [{"role": "assistant", "content": "How can I help you?"}]
langchain.debug = False

In [ ]:
#prompt = "Give me the premium for an insured sum of 10000000"
prompt = "The industry is chemical production"
#prompt = "And the temperature in Celsius?"
messages.append({"role": "user", "content": prompt})
print("user: " + prompt)
response = agent_executor.invoke({"input": prompt})
msg = {"role": "assistant", "content": response["output"]}
messages.append(msg)
print(textwrap.fill("assistant: " + response["output"], 80))
print("\nHistory:")
for msg in messages:
    #st.chat_message(msg["role"]).write(msg["content"])
    print(textwrap.fill(msg["role"] + ": " + msg["content"], 80))

In [ ]:
print("Memory from response object:")
chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
chat_history = chat_history_string.split("§§§ ")
msg_history = []
for chat_string in chat_history:
  tmp = chat_string.split("(content='")
  msg_history.append({"role": tmp[0], "content": tmp[1][:-2]})
for msg in msg_history:
  print(msg)

# CLI for agent (Chatbot with memory)

In [ ]:
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="chat_history")],
}
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=False,
    agent_kwargs=agent_kwargs,
    memory=memory,
)
langchain.debug = False

In [ ]:
agent.run("hi")
agent.run("My name is Bob")
agent.run("Wie lautet mein Name?")
agent.run("Give me the premium for an insured sum of 10000000.")
agent.run("it is metal construction industry")

'The premium for an insured sum of 10,000,000 in the metal construction industry is 2,500,000.'

# Work in progress...

In [ ]:
os.environ["LANGCHAIN_TRACING"] = "true"
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')

In [ ]:
# Tool 1
def multiplier(a: float, b: float) -> float:
    """Multiply the provided floats."""
    return a * b

tool = StructuredTool.from_function(multiplier)

agent_executor = initialize_agent(
    [tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
# The one Agent than can accept a structured tool with multiple arguments is the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION Agent type
# add memory: https://python.langchain.com/docs/modules/agents/agent_types/structured_chat#adding-in-memory

prompt = "What is 3 times 4"
response_text = agent_executor.run(prompt)
print(response_text)
response = agent_executor.invoke({"input": prompt})
#print(textwrap.fill(str(response["output"]), 80))


In [ ]:
# Tool 2
def calculate_insurance_premium(insured_sum: int, industry: str) -> float:
    """Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer."""
    premium = insured_sum*0.25
    return premium

tool = StructuredTool.from_function(calculate_insurance_premium, return_direct=False)

agent_executor = initialize_agent(
    [tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

prompt = "Give me the premium for an insured sum of 10000000"
prompt = "Kalkuliere die Versicherungsprämie für eine Versicherungssumme von 10000000, die Branche ist Metallbau"
response_text = agent_executor.run(prompt)
print(response_text)
response = agent_executor.invoke({"input": prompt})
#print(textwrap.fill(str(response["output"]), 80))


# Manual loop for agent_executor

## Test 1

Works only for single prompt - no memory!

In [ ]:
# see https://levelup.gitconnected.com/building-a-powerful-agent-has-no-challenge-today-774e27be818d

# only for single prompt - no memory !

def calculate_insurance_premium(insured_sum: int, industry: str) -> float:
    """Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer."""
    premium = insured_sum*0.25
    return premium

calc_tool = StructuredTool.from_function(calculate_insurance_premium, return_direct=False)

tools = [calc_tool]
tools_dict = {"calculate_insurance_premium": calc_tool}

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

langchain.debug = False

intermediate_steps = []
while True:
    output = agent.invoke({
        "input": "Kalkuliere die Versicherungsprämie für eine Versicherungssumme von 10000000, die Branche ist Metallbau.",
        "intermediate_steps": intermediate_steps
    })
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(output.tool, output.tool_input)
        #tool = {
        #    "get_word_length": get_word_length
        #}[output.tool]
        tool = tools_dict[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))

print(final_result)

calculate_insurance_premium {'insured_sum': 10000000, 'industry': 'Metallbau'}
Die Versicherungsprämie für eine Versicherungssumme von 10.000.000 in der Branche Metallbau beträgt 2.500.000.


## Test 2

Not working, memory implementation throws error

In [122]:
from langchain.schema import BaseMessage
hhh = BaseMessage(type=("User"), content=("test"))

In [ ]:
# see https://levelup.gitconnected.com/building-a-powerful-agent-has-no-challenge-today-774e27be818d

# only for single prompt - no memory !

def calculate_insurance_premium(insured_sum: int, industry: str) -> float:
    """Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer."""
    premium = insured_sum*0.25
    return premium

calc_tool = StructuredTool.from_function(calculate_insurance_premium, return_direct=False)

tools = [calc_tool]
tools_dict = {"calculate_insurance_premium": calc_tool}

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant."),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

langchain.debug = False

intermediate_steps = []
chat_history = []

myprompt = "Kalkuliere die Versicherungsprämie für eine Versicherungssumme von 10000000"

while True:
    output = agent.invoke({
        "input": myprompt,
        "intermediate_steps": intermediate_steps,
        "chat_history": memory
    })
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        # hier der Fehler? variable chat_history should be a list of base messages
        #msg = {"role": "user", "content": myprompt}
        chat_history.append(BaseMessage(type=("User"), content=(myprompt)))
        #msg = {"role": "assistant", "content": output.return_values["output"]}
        chat_history.append(BaseMessage(type=("assistant"), content=(output.return_values["output"])))
        break
    else:
        print(output.tool, output.tool_input)
        print(output)
        #tool = {
        #    "get_word_length": get_word_length
        #}[output.tool]
        tool = tools_dict[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))

print(final_result)

ValueError: ignored